PROBLEM 1

In [171]:
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from PIL import Image
from sklearn.datasets import fetch_openml

minst = fetch_openml('mnist_784', cache=True)


In [182]:
x = minst.data.astype('float32')
y = minst.target.astype('int64')
x['target'] = y
X = []
for i in range(10):
  X.append(x[x['target']==i])
  X[i] = X[i].sample(frac=1).reset_index(drop=True)

In [183]:
# split the data into training and testing
# train ratio is 70%
# test ratio is 15%
# validation ratio is 15%
# transform the data into a torch tensor and normalize it to be between 0 and 1
rowRange = y.value_counts()
train = []
test = []
validation = []
trainY = []
validationY = []
testY = []

def transform(array):
    array[i] = array[i].to_numpy()
    array[i] = torch.Tensor(array[i])
    array[i] = array[i]/255

def getSizes(index):
    trainSize = int(0.7 * len(X[index]))
    validationSize = int(0.15 * len(X[index]))
    testSize = len(X[index]) - trainSize - validationSize
    return trainSize, validationSize, testSize

for i in range(10):
    trainSize, validationSize, testSize = getSizes(i)
    train.append(X[i][:trainSize])
    validation.append(X[i][trainSize:trainSize+validationSize])
    test.append(X[i][trainSize+validationSize:])

    trainY.append(train[i]['target'].to_numpy())
    validationY.append(validation[i]['target'].to_numpy())
    testY.append(test[i]['target'].to_numpy())
    # cleanup
    del train[i]['target']
    del validation[i]['target']
    del test[i]['target']
    transform(train)
    transform(test)
    transform(validation)

    
    

Problem 2:

In [157]:
# Binary classification via KNN for 2 and 3 combination
# 2 is the positive class and 3 is the negative class
# implement the KNN algorithm
# use the Euclidean distance and the Manhattan distance
# use the KNN algorithm to classify the test data

# create data and label concatenation 2 and 3

data = torch.cat((train[2], train[3]), 0)
dataLabel = np.concatenate((trainY[2], trainY[3]), 0)
validation = torch.cat((validation[2], validation[3]), 0)
validationLabel = np.concatenate((validationY[2], validationY[3]), 0)


In [142]:
def norm(x,y, type):
  if(type=="L1"):
    return torch.norm(x-y, p=1, dim=1)
  elif(type=="L2"):
    return torch.norm(x-y, p=2, dim=1)

def knn(data, dataLabel, validation, validationLabel, k, distance_type):
  hitCount = 0
  for target in range(len(validation)):
    result = norm(data, validation[target], distance_type)
    # get the k smallest distances
    knn = result.topk(k, largest=False)
    count = Counter(dataLabel[knn.indices])
    hit = count.most_common(1)[0][0]
    if(hit == validationLabel[target]):
      hitCount += 1
  return round(hitCount/len(validation)*100, 2)

print("Problem 2 ANSWER (a)")
pred = knn(data, dataLabel, validation, validationLabel, 5, "L1")
pred

Problem 2 ANSWER (a)


99.34

#2 b) What are the hyperparameters you can tune?

In [ ]:
# b) What are the hyperparameters you can tune?
# k, distance type

c) Try at least two other options for each hyperparameter. Report the performance for
each option.

In [83]:
# c) Try at least two other options for each hyperparameter. Report the performance for
# each option.
print("Problem 2 ANSWER (c)")
for k in range(3,7,2):
    predL1 = knn(data, dataLabel, validation, validationLabel, k, "L1")
    predL2 = knn(data, dataLabel, validation, validationLabel, k, "L2")
    print("K ", k)
    print("L1 (Manhatten Distance): ", predL1)
    print("L2 (Euclidean Distance): ", predL2)


Problem2 - (c)
K  3
L1  99.48
L2  99.67
K  5
L1  99.34
L2  99.43


d) You can try more options if you want. What is the final test accuracy?

In [109]:
# d) You can try more options if you want. What is the final test accuracy?
predL2 = knn(data, dataLabel, validation, validationLabel, 3, "L2")
print("K ", 3)
print("L2 (Euclidean Distance): ", predL2)

K  3
L2 (Euclidean Distance):  99.48


Problem #3: Multiclass Classification via k-NN [15pts]

a) Make a k-NN (k=5) and its training/validation/evaluation code to perform multiclass
classification over all digits.

In [112]:
# a) Make a k-NN (k=5) and its training/validation/evaluation code to perform multiclass
# classification over all digits.
data = torch.cat((train[0], train[1]), 0)
dataLabel = np.concatenate((trainY[0], trainY[1]), 0)
validation = torch.cat((validation[0], validation[1]), 0)
validationLabel = np.concatenate((validationY[0], validationY[1]), 0)

In [121]:
print("Problem 3 ANSWER (a)")
Pred = knn(data, dataLabel, validation, validationLabel, 5, "L2")
print("Multiclass KNN Accuracy: ", Pred)

Problem 3 ANSWER (a)
Multiclass KNN Accuracy:  99.2


b) What are the hyperparameters you can tune?

In [ ]:
# b) What are the hyperparameters you can tune?
# k, distance type

c) Try at least two other options for each hyperparameter. Report the performance for
each option.

In [122]:
# c) Try at least two other options for each hyperparameter. Report the performance for
# each option.
print("Problem 3 ANSWER (c)")
for k in range(3,7,2):
    predL1 = knn(data, dataLabel, validation, validationLabel, k, "L1")
    predL2 = knn(data, dataLabel, validation, validationLabel, k, "L2")
    print("K ", k)
    print("L1 (Manhatten Distance): ", predL1)
    print("L2 (Euclidean Distance): ", predL2)

Problem 3 ANSWER (c)
K  3
L1 (Manhatten Distance):  99.06
L2 (Euclidean Distance):  99.15
K  5
L1 (Manhatten Distance):  99.15
L2 (Euclidean Distance):  99.2


In [127]:
print("Problem 3 ANSWER (d)")
# d) You can try more options if you want. What is the final test accuracy?
predL2 = knn(data, dataLabel, validation, validationLabel, 3, "L2")
print("K ", 3)
print("Multiclass L2 (Euclidean Distance) (AUC): ", predL2)

Problem 3 ANSWER (d)
K  3
Multiclass L2 (Euclidean Distance) (AUC):  99.15


In [143]:
import torch.nn as nn
import torch.optim as optim

In [144]:
class HingeLoss(nn.Module):
    def __init__(self):
        super(HingeLoss, self).__init__()

    def forward(self, y, y_pred):
        return torch.mean(torch.clamp(1 - y * y_pred, min=0))

In [150]:
# def accuracy(X, Y, model):
def SVMAccuracy(X, Y, model):
    with torch.no_grad():
        y_pred = model(X)
        y_pred[y_pred >= 0] = 1
        y_pred[y_pred < 0] = -1
        accuracy = (y_pred == Y).sum().item() / len(Y)
    return accuracy


In [179]:
# def svm(X, Y, val, val_label, learning_rate, epoch, batch_size)
def svm(X, Y, val, val_label, learning_rate, epoch, batch_size):
    model = nn.Linear(784, 1)
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9)
    criterion = HingeLoss()
    model.train()
    for i in range(epoch):
        for j in range(0, len(X), batch_size):
            optimizer.zero_grad()
            y_pred = model(X[j:j+batch_size])
            loss = criterion(Y[j:j+batch_size], y_pred)
            loss.backward()
            optimizer.step()
        if(i%10 == 0):
            print("Epoch: ", i, " Loss: ", loss.item(), " Accuracy: ", SVMAccuracy(val, val_label, model))
    return model

In [184]:
X = torch.cat((train[2], train[3]), 0)
Y = np.concatenate((trainY[2], trainY[3]), 0)
Y[Y==2] = -1
Y[Y==3] = 1
validation = torch.cat((validation[2], validation[3]), 0)
validationLabel = np.concatenate((validationY[2], validationY[3]), 0)

In [186]:
print("Problem4 - (b)")
# HAVE Problem at my computer to run
svm(X, Y, validation, validationLabel, 0.1, 10, 64)

Problem4 - (b)


TypeError: unsupported operand type(s) for *: 'numpy.ndarray' and 'Tensor'

In [187]:
# Problem #4: Binary Classification via soft-margin SVM [15pts]
# had a problem running this